In [1]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np
from collections import OrderedDict

In [2]:
graph1 = gs.import_onnx(onnx.load('../orin_onnx/encoder.onnx'))
graph2 = gs.import_onnx(onnx.load('encoder_quant.onnx'))

In [3]:
Conv_1 = [node for node in graph1.nodes if node.name=='Conv_188'][0]
Conv_1

Conv_188 (Conv)
	Inputs: [
		Variable (input.51): (shape=None, dtype=None)
		Constant (onnx::Conv_3273): (shape=[256, 1, 15], dtype=<class 'numpy.float32'>)
		Constant (onnx::Conv_3274): (shape=[256], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (input.59): (shape=None, dtype=None)
	]
Attributes: OrderedDict([('dilations', [1]), ('group', 256), ('kernel_shape', [15]), ('pads', [7, 7]), ('strides', [1])])

In [4]:
Conv_2 = [node for node in graph2.nodes if node.name=='Conv_188'][0]
Conv_2

Conv_188 (Conv)
	Inputs: [
		Variable (PPQ_Variable_274): (shape=None, dtype=None)
		Variable (PPQ_Variable_280): (shape=None, dtype=None)
		Constant (onnx::Conv_3274): (shape=[256], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (input.59): (shape=None, dtype=None)
	]
Attributes: OrderedDict([('dilations', [1]), ('group', 256), ('kernel_shape', [15]), ('pads', [7, 7]), ('strides', [1])])

In [5]:
Conv_1.inputs[1]

Constant (onnx::Conv_3273): (shape=[256, 1, 15], dtype=<class 'numpy.float32'>)
LazyValues (shape=[256, 1, 15], dtype=float32)

In [6]:
Conv_2.inputs[1]

Variable (PPQ_Variable_280): (shape=None, dtype=None)

In [7]:
weights = Conv_1.inputs[1]

In [8]:
weights.values

array([[[-0.06179988, -0.11587369, -0.0782654 , ...,  0.09398152,
         -0.04613826,  0.07503603]],

       [[-0.03725612, -0.00824413,  0.00494244, ..., -0.00534191,
          0.00701523,  0.01457752]],

       [[-0.05152623, -0.05021266, -0.04398715, ...,  0.0163779 ,
         -0.00225453, -0.04670919]],

       ...,

       [[-0.01983024, -0.02094599, -0.0487788 , ...,  0.0498136 ,
         -0.00630857,  0.05127599]],

       [[ 0.00389191, -0.04586082, -0.01248292, ...,  0.03666314,
         -0.01951113,  0.02056173]],

       [[ 0.03362665,  0.04754938,  0.0577562 , ..., -0.01154593,
          0.02412693, -0.04146174]]], dtype=float32)

In [9]:
ppq = Conv_2.inputs[1]

In [10]:
ppq

Variable (PPQ_Variable_280): (shape=None, dtype=None)

In [11]:
tmp = Conv_1.inputs[1].copy()

In [12]:
tmp

Constant (onnx::Conv_3273): (shape=(256, 1, 15), dtype=<class 'numpy.float32'>)
[[[-0.06179988 -0.11587369 -0.0782654  ...  0.09398152 -0.04613826
    0.07503603]]

 [[-0.03725612 -0.00824413  0.00494244 ... -0.00534191  0.00701523
    0.01457752]]

 [[-0.05152623 -0.05021266 -0.04398715 ...  0.0163779  -0.00225453
   -0.04670919]]

 ...

 [[-0.01983024 -0.02094599 -0.0487788  ...  0.0498136  -0.00630857
    0.05127599]]

 [[ 0.00389191 -0.04586082 -0.01248292 ...  0.03666314 -0.01951113
    0.02056173]]

 [[ 0.03362665  0.04754938  0.0577562  ... -0.01154593  0.02412693
   -0.04146174]]]

In [13]:
tmp.name = 'noquant_weights'

In [14]:
Conv_2.inputs[1] = tmp

In [15]:
graph2.cleanup().toposort()
onnx.save(gs.export_onnx(graph2),'encoder_fix_quant.onnx')